In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [2]:
from sklearn import preprocessing 
from sklearn.metrics import pairwise 
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster 
from sklearn.cluster import KMeans 
from pandas.plotting import parallel_coordinates

In [13]:
ksi_df = pd.read_csv('Data/KSI.csv')

In [4]:
pd.set_option('max_columns', None)

In [ ]:
ksi_df

In [ ]:
ksi_df.info()

In [12]:
# making small function to collapse all drinking conditions into one

def drivcond(col):
    if col['DRIVCOND'] in ('Ability Impaired, Alcohol Over .08', 'Had Been Drinking', 'Ability Impaired, Alcohol' ,'Ability Impaired, Drugs'):
        return 'Ability Impaired'
    if col['DRIVCOND'] in ('Inattentive', 'Fatigue'):
        return 'Inattentive/Fatigued'
    # if not drinking, return condition
    
    
    return col['DRIVCOND']

In [6]:
def actcollapse(col):
    if col['DRIVACT'] in ('Improper Turn', 'Improper Passing'):
        return 'Improer Turn/Passing'
    if col['DRIVACT'] in ('Speed too Fast For Condition', 'Exceeding Speed Limit'):
        return 'Speeding'
    return col['DRIVACT']

def mancollapse(col):
    if col['MANOEUVER'] in ('Turning Left', 'Turning Right'):
        return 'Turning'
    if col['MANOEUVER'] in ('Slowing or Stopping', 'Stopped', 'Parked'):
        return 'Stopped/Stopping'
    return col['MANOEUVER']

In [14]:
ksi_df = ksi_df.iloc[ksi_df['ACCNUM'].drop_duplicates().index]

predictors = ksi_df.iloc[:,40:53]

In [15]:
predictors = ksi_df[['DRIVACT','DRIVCOND','MANOEUVER']]
predictors['DRIVCOND'] = predictors.apply(drivcond, axis = 1)
predictors['DRIVACT'] = predictors.apply(actcollapse, axis = 1)
predictors['MANOEUVER'] = predictors.apply(mancollapse, axis = 1)
predictors = pd.get_dummies(predictors)

<ipython-input-15-f22760f48c9d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictors['DRIVCOND'] = predictors.apply(drivcond, axis = 1)
<ipython-input-15-f22760f48c9d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictors['DRIVACT'] = predictors.apply(actcollapse, axis = 1)
<ipython-input-15-f22760f48c9d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [17]:
predictors.head()

,DRIVACT_Disobeyed Traffic Control,DRIVACT_Driving Properly,DRIVACT_Failed to Yield Right of Way,DRIVACT_Following too Close,DRIVACT_Improer Turn/Passing,DRIVACT_Improper Lane Change,DRIVACT_Lost control,DRIVACT_Speed too Slow,DRIVACT_Speeding,DRIVACT_Wrong Way on One Way Road,DRIVCOND_Ability Impaired,DRIVCOND_Inattentive/Fatigued,DRIVCOND_Medical or Physical Disability,DRIVCOND_Normal,MANOEUVER_Changing Lanes,MANOEUVER_Disabled,MANOEUVER_Going Ahead,MANOEUVER_Making U Turn,MANOEUVER_Merging,MANOEUVER_Overtaking,MANOEUVER_Pulling Away from Shoulder or Curb,MANOEUVER_Pulling Onto Shoulder or towardCurb,MANOEUVER_Reversing,MANOEUVER_Stopped/Stopping,MANOEUVER_Turning
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0


In [16]:
predictors = predictors[[col for col in predictors.columns if col not in ('DRIVACT_<Null>','DRIVACT_Other','DRIVACT_Unknown', 'DRIVCOND_<Null>','DRIVCOND_Other','DRIVCOND_Unknown', 'MANOEUVER_<Null>','MANOEUVER_Other','MANOEUVER_Unknown')]]

In [ ]:
predictors  = predictors.replace('Yes', 1).replace('<Null>', 0).replace('<Null', 0)

In [ ]:
predictors = predictors[[col for col in predictors.columns if col not in ('TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH', 'AUTOMOBILE')]]

In [18]:
# don't attempt to run this on your local pc
# this will crash your kernel
plt.figure(figsize = (40, 40))
Z = linkage(predictors, method= 'single') 
dendrogram(Z, labels= predictors.index)